# Understanding and managing LLM short-term memory

- Large language models are stateless; each invocation is independent
  - Models do not remember what was sent in the previous message
  - Models do not "learn" from a conversation while they are in inference mode
- But, chat conversations appear to have memory because conversation history is provided as "context"
- Deciding what to provide as historical context is the "art" of LLM memory management
- `LangChain` provides different kinds of "memory" to store and accumulate the context of a conversation

In [ ]:
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langchain.messages import HumanMessage
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)
from langchain_core.runnables.history import RunnableWithMessageHistory
from langgraph.checkpoint.memory import InMemorySaver

from chain_reaction.config import APIKeys, ModelName
from chain_reaction.memory import ChatSessionHistoryManager

In [ ]:
# Load API keys from .env file
api_keys = APIKeys()

# Initialize a chat model with your API key
chat_model = init_chat_model(
    model=ModelName.CLAUDE_HAIKU,
    temperature=0,
    max_tokens=1024,
    timeout=None,
    max_retries=2,
    api_key=api_keys.anthropic,
)

# A forgetful chatbot

In [ ]:
# Initialize prompt template with system message and placeholders for chat history
system_prompt = SystemMessagePromptTemplate.from_template(
    "You are a friendly and helpful chat model. "
    "The most important thing is to always remember the user's name and use it in the conversation. "
    "If you don't know the user's name, ask for it politely."
)

prompt_template = ChatPromptTemplate.from_messages([
    system_prompt,
    MessagesPlaceholder(variable_name="chat_history"),
    HumanMessagePromptTemplate.from_template("{user_input}"),
])

In [ ]:
# Initialize the chat session history manager with a limit of 0 message (forgetful)
chat_session_manager = ChatSessionHistoryManager(max_messages=0)
session_config = chat_session_manager.create_session_config()

In [ ]:
# Define the runnable pipeline with message history
pipeline = prompt_template | chat_model
pipeline_with_history = RunnableWithMessageHistory(
    pipeline,
    get_session_history=chat_session_manager.get_chat_history,
    input_messages_key="user_input",
    history_messages_key="chat_history",
)

In [ ]:
# Turn 1: User provides their name and asks for a joke
response = pipeline_with_history.invoke(
    {"user_input": "My name is Nick, tell me a joke about pydantic."},
    config=session_config,
)
print(response.content)

In [ ]:
# Turn 2: User reacts to the joke and asks to be reminded of their name
response2 = pipeline_with_history.invoke(
    {"user_input": "Wow, that's funny! Can you remind me what my name is?"},
    config=session_config,
)
print(response2.content)

In [ ]:
"Nick" in response2.content

In [ ]:
# Turn 3: User gives their name again and asks for a joke
response3 = pipeline_with_history.invoke(
    {
        "user_input": (
            "No problem, my name is Nick, please don't forget it this time! Tell me a joke about short-term memory."
        )
    },
    config=session_config,
)
print(response3.content)

In [ ]:
# Turn 4: User checks if the model remembers their name
response4 = pipeline_with_history.invoke(
    {"user_input": "Good one! Ok, this should be easy now, what's my name?"},
    config=session_config,
)
print(response4.content)

In [ ]:
"Nick" in response4.content

# Agent with memory

In [ ]:
# Initialize an agent with memory
agent = create_agent(
    model=chat_model,
    checkpointer=InMemorySaver(),
    system_prompt=system_prompt.prompt.template,
)

# Initialize configuration for a new chat session
config = {"configurable": {"thread_id": 1}}

# Prompt agent
response = agent.invoke(
    input={"messages": [HumanMessage("My name is Nick, tell me a joke about pydantic.")]},
    config=config,
)

In [ ]:
response_2 = agent.invoke(
    input={"messages": [HumanMessage("Wow, that's funny! Can you remind me what my name is?")]},
    config=config,
)
response_2["messages"][-1].content